In [40]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import math
import time
#import rasterio
#from rasterio import features

In [36]:
file = 'gpd.csv'

 

df = pd.read_csv(file)

 

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

wkt_geom = [i.wkt for i in gdf.geometry]
#gdf.explore()

In [29]:

#Setup working spatial reference
sr_wkt = 'LOCAL_CS["arbitrary"]'
sr = osr.SpatialReference( sr_wkt )

pix_size = 0.0000001

x_0 = -89.7797378
y_0 =  40.0303053
# Create a memory raster to rasterize into.

target_ds = gdal.GetDriverByName('MEM').Create( '',10000, 33000, 3,gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))
target_ds.SetProjection(sr_wkt)
 

# Create a memory layer to rasterize from.

rast_ogr_ds = ogr.GetDriverByName('Memory').CreateDataSource( 'wrk' )
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr )

 

for g in wkt_geom:
    feat = ogr.Feature( rast_mem_lyr.GetLayerDefn() )
    feat.SetGeometryDirectly( ogr.Geometry(wkt = g) )
    rast_mem_lyr.CreateFeature( feat )

 

# Run the algorithm.

 

gdal.RasterizeLayer(target_ds, [3,2,1], rast_mem_lyr, burn_values = [4095,4095,4095], options = ["BURN_VALUE_FROM=Z"])
gdal.GetDriverByName('GTiff').CreateCopy('rasterize.tif',target_ds)

 


print('success')

success


In [37]:
rast_ogr_ds

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x0000019CAD82F390> >

In [41]:
wkt_geom = []
for i in gdf.geometry:
    wkt_geom.append(i.wkt)

 

x_size = gdf.total_bounds[2]-gdf.total_bounds[0]
y_size = gdf.total_bounds[3]-gdf.total_bounds[1]
x_0 = gdf.total_bounds[0]
y_0 = gdf.total_bounds[3]

 

pix_size = 0.000001
pixels_in_x = math.ceil(x_size/pix_size)
pixels_in_y = math.ceil(y_size/pix_size)

 

t1 = time.time()

 

# Setup working spatial reference
sr_wkt = 'LOCAL_CS["arbitrary"]'
sr = osr.SpatialReference(sr_wkt)

 


sr = osr.SpatialReference()
sr.ImportFromEPSG(4326)
sr.SetProjection ("EPSG:4326")
sr_wkt = sr.ExportToWkt()
#print (sr_wkt)

 

# Create a memory raster to rasterize into.

 

target_ds = gdal.GetDriverByName('MEM').Create('', pixels_in_x, pixels_in_y, 3, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))
target_ds.SetProjection(sr_wkt)

 

# Create a memory layer to rasterize from.

 

rast_ogr_ds  = ogr.GetDriverByName('Memory').CreateDataSource( 'wrk' )
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr)

 

# Add polygons and linestrings.
wkt_geom = wkt_geom
#wkt_geom = ['POLYGON((0 0 0,0 10 0,10 10 0,10 0 0,0 0 0))']

 

for g in wkt_geom:
    feat = ogr.Feature(rast_mem_lyr.GetLayerDefn())
    feat.SetGeometryDirectly(ogr.Geometry(wkt = g))
    rast_mem_lyr.CreateFeature(feat)

 

gdal.RasterizeLayer(target_ds, [1,2,3], rast_mem_lyr, burn_values = [0,255,0])
#gdal.GetDriverByName('PNG').CreateCopy('rasterize.png',target_ds)

 

tiffdriver = gdal.GetDriverByName('GTiff')
tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

 

t2 = time.time()
print(str(round(t2-t1,2)) + ' segundos')

0.31 segundos


In [50]:
sr_wkt

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [7]:
import platform
#print(platform.system(),)
#print(platform.machine(),)
print(platform.platform(),)
print(platform.uname(),)
print(platform.version(),)
print(platform.mac_ver(),)

Windows-10-10.0.19045-SP0
uname_result(system='Windows', node='C118', release='10', version='10.0.19045', machine='AMD64')
10.0.19045
('', ('', '', ''), '')
